In [10]:
import findspark
findspark.init()
from pyspark import SparkContext
import pickle
from numpy import linalg as LA
sc.stop()
sc = SparkContext(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark.sql import *
sqlContext = SQLContext(sc)

from time import time
N=sc.defaultParallelism
print 'Number of executors=',N

 Number of executors= 8


In [11]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [12]:
def index_to_letters(n):
    Letters=['B','S']
    out=''
    for i in range(8):
        out+=Letters[n%2]
        n=n/2
    return out
index_to_letters(201)

'SBBSBBSS'

In [ ]:
#file_index='BBBSBBBB'
data_dir='../../Data/Weather'

for index in range(256):
    
    file_index=index_to_letters(index)

    filebase='US_Weather_%s'%file_index
    !rm -rf $data_dir/$filebase*

    c_filename=filebase+'.csv.gz'
    u_filename=filebase+'.csv'

    command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(c_filename,data_dir,c_filename)
    print command
    !$command
    !ls -lh $data_dir/$c_filename

    #unzip
    !gunzip -c $data_dir/$c_filename > $data_dir/$u_filename
    List=pickle.load(open(data_dir+'/'+u_filename,'rb'))
    len(List)

    df=sqlContext.createDataFrame(List)
    print df.count()

    measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

    sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

    from numpy import linalg as LA
    STAT={}  # dictionary storing the statistics for each measurement
    Clean_Tables={}

    for meas in measurements:
        t=time()
        Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
        print Query
        df = sqlContext.sql(Query)
        data=df.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
        #get very basic statistics
        STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

        # compute covariance matrix
        OUT=computeCov(data)

        #find PCA decomposition
        eigval,eigvec=LA.eig(OUT['Cov'])

        # collect all of the statistics in STAT[meas]
        STAT[meas]['eigval']=eigval
        STAT[meas]['eigvec']=eigvec
        STAT[meas].update(OUT)

        print 'time for',meas,'is',time()-t

    filename=data_dir+'/STAT_%s.pickle'%file_index
    pickle.dump((STAT,STAT_Descriptions),open(filename,'wb'))
    break

curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_BBBBBBBB.csv.gz > ../../Data/Weather/US_Weather_BBBBBBBB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4769k  100 4769k    0     0  2771k      0  0:00:01  0:00:01 --:--:-- 2772k
-rw-r--r--  1 yoavfreund  staff   4.7M Apr 30 09:45 ../../Data/Weather/US_Weather_BBBBBBBB.csv.gz
12854
SELECT * FROM weather
	WHERE measurement = 'TMAX'
shape of E= (365,) shape of NE= (365,)
time for TMAX is 16.2257618904
SELECT * FROM weather
	WHERE measurement = 'SNOW'
shape of E= (365,) shape of NE= (365,)
time for SNOW is 6.27778887749
SELECT * FROM weather
	WHERE measurement = 'SNWD'
shape of E= (365,) shape of NE= (365,)
time for SNWD is 4.80610918999
SELECT * FROM weather
	WHERE measurement = 'TMIN'
